## Eigenfaces - Face Recognition

#### Importing necessary libraries

In [2]:
import os
import numpy as np
from PIL import Image

#### Loading and transforming the images into vectors

In [14]:
def load_images(path):
    images = {}
    for file in filter(lambda file: not file.startswith('.'), os.listdir(path)):
        img = Image.open(os.path.join(path, file))
        idx = int(file.split('_')[0])
        if not idx in images:
            images[idx] = []
        images[idx].append(np.asarray(img))

    return images

def transform_to_vec(dataset):
    for person_face in dataset:
        dataset[person_face] = map(lambda img: img.flatten(), dataset[person_face])
    return dataset

In [16]:
DATA_PATH = '../data'
TRAIN_DATA_PATH = os.path.join(DATA_PATH, 'train')
TEST_DATA_PATH = os.path.join(DATA_PATH, 'test')

train_data = load_images(TRAIN_DATA_PATH)
test_data = load_images(TEST_DATA_PATH)

train_data = transform_to_vec(train_data)
test_data = transform_to_vec(test_data)


/Users/gabriel-rusu/Desktop/projects/github/eigenfaces/src
/Users/gabriel-rusu/Desktop/projects/github/eigenfaces/src
